# Heroes of Pymoli

In [ ]:
# import pandas
import pandas as pd

In [ ]:
# Load the file

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

### Player Count

In [ ]:
# Count the number of Players (which is the number of rows in dataset)
totalPlayers = purchase_data["SN"].value_counts().count()
# Display it as data frame
totalPlayersDF = pd.DataFrame({"Total Players":[ totalPlayers]})
totalPlayersDF

### Purchasing Analysis (Total)

In [ ]:
# Number of unique items
totalUniqueItems = purchase_data["Item Name"].value_counts().count()
# Average purchase price
averagePurchasePrice = round(purchase_data["Price"].mean(),2)
# Total number of Purchases
totalNumberPurchases = purchase_data["Purchase ID"].count()
# Total Revenue
totalRevenue = purchase_data["Price"].sum()
PurchaseResults = pd.DataFrame({
    "Number of Unique Items":[totalUniqueItems],
    "Average Price": [averagePurchasePrice],
    "Number of Purchases": [totalNumberPurchases],
    "Total Revenue":[totalRevenue]
    
})
PurchaseResults["Average Price"] = PurchaseResults["Average Price"].map("${:.2f}".format)
PurchaseResults["Total Revenue"] = PurchaseResults["Total Revenue"].map("${:,.2f}".format)
PurchaseResults

### Gender Demographics

In [ ]:
# Store unique players and their related info, (SN, Age, Gender) into dataframe
PlayerInfo = purchase_data.iloc[:,1:4]
uniquePlayersInfo = PlayerInfo.drop_duplicates(["SN","Gender"])
uniquePlayersInfoG = uniquePlayersInfo.groupby("Gender")
# #Find the counts of each gender

genderCounts = uniquePlayersInfoG.count()
genderCounts = pd.DataFrame(genderCounts["SN"])

# Store each gender count into its own variable
femaleCount = genderCounts.loc["Female"]
maleCount = genderCounts.loc["Male"]
otherCount = genderCounts.loc["Other / Non-Disclosed"]

malePercent = round((maleCount / totalPlayers) * 100,2)
femalePercent = round((femaleCount / totalPlayers) * 100,2)
otherPercent = round((otherCount / totalPlayers) * 100,2)

genderResults = pd.DataFrame({
    "Total Count": [maleCount[0], femaleCount[0], otherCount[0]],
    "Percentage of Players" : [malePercent[0],femalePercent[0], otherPercent[0]],
    "Gender" : ["Male", "Female","Other / Non-Disclosed"]
})

genderResults["Percentage of Players"] = genderResults["Percentage of Players"].map("{:.2f}%".format)
genderResults = genderResults.set_index("Gender")
genderResults

### Purchasing Analysis (Gender)

In [ ]:
# Group by Gender
# Count how many purchases each gender made
purchGend = purchase_data[["Gender","SN"]].groupby("Gender").count()

# Rename SN column to Purchase Count
purchGend = purchGend.rename(columns = {"SN": "Purchase Count"})

# Average purchase price each for gender
purchGend["Average Purchase Price"] =purchase_data[["Gender","Price"]].groupby("Gender").mean()

# Add Total purchase price for each gender
purchGend["Total Purchase Value"] = purchase_data[["Gender","Price"]].groupby("Gender").sum()
purchGend

# Add Avergae total purchase per person
# Group by gender with name and price 
groupGender = purchase_data.groupby(["Gender","SN"])["Price"]
# Average Total Purchae per person for each gender
# Female
femaleMean = groupGender.sum().loc["Female"].mean()
# Male
maleMean = groupGender.sum().loc["Male"].mean()
# Other
othermean = groupGender.sum().loc["Other / Non-Disclosed"].mean()

purchGend["Avg Total Purrchase per Person"] = [femaleMean,maleMean,othermean]

# Format Data Frame
purchGend["Average Purchase Price"]= purchGend["Average Purchase Price"].map("${:,.2f}".format)
purchGend["Total Purchase Value"]= purchGend["Total Purchase Value"].map("${:,.2f}".format)
purchGend["Avg Total Purrchase per Person"]= purchGend["Avg Total Purrchase per Person"].map("${:,.2f}".format)

# Display Summary
purchGend

### Age Demographics

In [ ]:
# Remove duplicate player names
playerUnique = purchase_data.drop_duplicates(["SN","Age"])


In [ ]:
# create bins
bins = [0,9,14,19,24,29,34,39,50]
# create labels
binLabels = ["<10", "10-14", "15-19","20-24","25-29","30-34","35-39","40+"]
#Place data into bins
binData = pd.cut(playerUnique["Age"],bins, labels = binLabels)
binData.value_counts()
# # add this column to dataframe
binData = pd.Series(binData, name ="Age Bracket")

ageBracket = pd.concat([playerUnique,binData],axis = 1)

# ageBracket[ageBracket["Age Bracket"].isnull()]
result = pd.DataFrame(ageBracket["Age Bracket"].value_counts()).sort_index()
result["Percentage of Players"] = (result["Age Bracket"] / totalPlayers)*100
result["Percentage of Players"] = result["Percentage of Players"].map("{:.2f}%".format)
result

### Purchasing Analysis (Age)

In [ ]:
binData2 = pd.cut(purchase_data["Age"],bins, labels = binLabels)
purchase_data["Age Bracket"] = binData2
# Total Count
AgeDemographic = pd.DataFrame(purchase_data.groupby("Age Bracket")["SN"].count())

# # Average Purchase Price
avgPurchase = purchase_data.groupby("Age Bracket")["Price"].mean()

# # Total Purchase value
totalPurchase = purchase_data.groupby("Age Bracket")["Price"].sum()


# Average Total Purchae per person for each Age Bracket
# Group by age bracket
age = purchase_data.groupby(["Age Bracket","SN"])

# Sums the total amount purchased by each player for watch
age2 = age.sum()

# Remove all rows that sum to zero (this means players werent in that age bracket)
age3 = age2[age2["Price"] != 0]

meanList = []

# find mean for each group
for name in binLabels:
    mean = age3["Price"].loc[name].mean()
    meanList.append(mean)

# Merge all results into one data frame
AgeDemographic["Average Purchase Price"] = avgPurchase
AgeDemographic["Total Purchase Value"] = totalPurchase
AgeDemographic["Avg Total Purchase per Person"] = meanList

# Format results
AgeDemographic["Average Purchase Price"] =AgeDemographic["Average Purchase Price"].map("${:.2f}".format)
AgeDemographic["Total Purchase Value"] =AgeDemographic["Total Purchase Value"].map("${:.2f}".format)
AgeDemographic["Avg Total Purchase per Person"] =AgeDemographic["Avg Total Purchase per Person"].map("${:.2f}".format)
# Print result
AgeDemographic


In [60]:
# Group player
groupPlayer = purchase_data.groupby("SN")["Price"]

# Total spent 
topSpenders = pd.DataFrame(groupPlayer.sum())

# Find top 5 spenders
topSpenders = topSpenders.sort_values(by = "Price",ascending = False).head(5)
names = topSpenders.index
topSpenders
 
# ind average purchase price
t5means = groupPlayer.mean().loc[names]

# topSpenders["Average Purchase Price"] = counts
# Total count of purchases for top 5
t5counts = groupPlayer.count().loc[names]
t5counts
# Create output, rename columns 
topSpenders["Purchase Count"] = t5counts
topSpenders["Average Purchase Price"] = t5means
topSpenders = topSpenders.rename(columns = {"Price": "Total Purchase Value"})

# change the order
topSpenders = topSpenders[["Purchase Count","Average Purchase Price","Total Purchase Value"]]

# change the format
topSpenders["Average Purchase Price"] = topSpenders["Average Purchase Price"].map("${:,.2f}".format)
topSpenders["Total Purchase Value"] = topSpenders["Total Purchase Value"].map("${:,.2f}".format)

# Print the output
topSpenders

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
